# 1) Clean Spotify Streaming History and MyLibrary CSV files

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
import downloading_and_cleaning as dc
from dotenv import load_dotenv
load_dotenv()
client_ID = os.getenv("client_ID")
client_secret = os.getenv("client_secret")

### Spotify provides the streaming history into two separeted CSV files that need merger.

In [2]:
stream0 = pd.read_json('data/Raw_Data/StreamingHistory0.json')
stream1 = pd.read_json('data/Raw_Data/StreamingHistory1.json')
df_stream = pd.concat([stream0, stream1])
df_stream['endTime'] = pd.to_datetime(df_stream['endTime'])
df_stream.sample(3)

,endTime,artistName,trackName,msPlayed
1971,2021-05-29 21:44:00,Red Hot Chili Peppers,Porcelain,163457
241,2021-05-02 18:49:00,Colapesce,Totale,2043
2492,2021-06-06 14:33:00,Rancid,Hoover Street,249733


#### Split time into multiple columns and transformed msPlayed (milisecs.) to minutes.

In [3]:
df_stream['year'] = pd.DatetimeIndex((df_stream.endTime)).year
df_stream['month'] = pd.DatetimeIndex((df_stream.endTime)).month
df_stream['day'] = pd.DatetimeIndex((df_stream.endTime)).day
df_stream['hour'] = pd.DatetimeIndex((df_stream.endTime)).hour
df_stream['minutesPlayed'] = round(df_stream['msPlayed']/60000,2)
df_stream.sample(3)

,endTime,artistName,trackName,msPlayed,year,month,day,hour,minutesPlayed
154,2021-05-01 17:13:00,Del The Funky Homosapien,Lyric Licking,15418,2021,5,1,17,0.26
1941,2020-12-07 19:43:00,Face To Face,Bent but Not Broken,150547,2020,12,7,19,2.51
4190,2021-02-03 22:38:00,Lagwagon,Reign,3552,2021,2,3,22,0.06


### The key that will enable the iterations with  Spotify's API is contained in another file called "MyLibrary".
#### The key name is "uri" and referes to the "track_id".

In [4]:
library0 = open('data/Raw_Data/YourLibrary.json')
library1 = json.load(library0)
df_library = pd.DataFrame(library1['tracks'])
df_library['uri'] = df_library.uri.str.split('spotify:track:', expand=True)[1]
df_library.sample(3)

,artist,album,track,uri
685,Sugarcult,Punk Goes Acoustic,Memory,5GMOO1d22BsIvMmlhxUwrg
244,The Low Life,Daisy Cutter,Castaway,7G2qfNCNMxL4LYdNL3cWtF
577,Metronomy,The English Riviera,The Bay,3ySxo6vRP0t9WCqhYXXMWN


### Merger of the StreamingHistory and MyLibrary to obtain the "Track_ID".
#### This is the last step to finish cleaning the CSV files provided by Spotify.

In [5]:
df_stream = df_stream.merge(df_library, how='inner', 
                              left_on=['artistName', 'trackName'],
                              right_on=['artist', 'track'])

df_stream = df_stream[['endTime', 'year', 'month', 'day', 'hour', 'artist', 'album', 'track', 'minutesPlayed', 'uri']]
df_stream.sample(3)

,endTime,year,month,day,hour,artist,album,track,minutesPlayed,uri
131,2021-02-21 23:16:00,2021,2,21,23,The Cure,Staring At The Sea - The Singles,Boys Don't Cry,2.59,1YQCRXscu8xE74BVVMUCRr
97,2020-10-31 20:17:00,2020,10,31,20,The Beatles,Anthology 2,It's Only Love - Anthology 2 Version,1.97,7EwuWi1xtmBLWYxWIyUj0r
514,2021-09-07 18:04:00,2021,9,7,18,Coolio,It Takes a Thief,"Mama, I'm in Love Wit a Gangsta - Mix",1.17,62h3HmtlAmot4A9zwKVRSq


# 2) Connect to Spotify API

In [10]:
token , headers = dc.connect_api(client_ID, client_secret)

# 3) Iterate with the API

In [7]:
uris = list(df_library['uri'].unique())

In [8]:
df_features = dc.song_attributes('audio-features/', uris, headers)
df_features.head(3)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.632,0.471,0,-7.887,1,0.0348,0.313,0.000005,0.0712,0.764,121.672,audio_features,5e8uc7f0v5jpY5SF1emxHl,spotify:track:5e8uc7f0v5jpY5SF1emxHl,https://api.spotify.com/v1/tracks/5e8uc7f0v5jp...,https://api.spotify.com/v1/audio-analysis/5e8u...,168120,4
1,0.875,0.481,1,-13.337,1,0.3120,0.134,0.001120,0.1320,0.836,85.021,audio_features,62h3HmtlAmot4A9zwKVRSq,spotify:track:62h3HmtlAmot4A9zwKVRSq,https://api.spotify.com/v1/tracks/62h3HmtlAmot...,https://api.spotify.com/v1/audio-analysis/62h3...,250160,4
2,0.712,0.676,9,-7.313,0,0.1530,0.191,0.066500,0.1250,0.869,79.994,audio_features,0HmQkmfYXRPweIg1ycw1R8,spotify:track:0HmQkmfYXRPweIg1ycw1R8,https://api.spotify.com/v1/tracks/0HmQkmfYXRPw...,https://api.spotify.com/v1/audio-analysis/0HmQ...,291440,4


In [11]:
df_tracks = dc.song_attributes('tracks/', uris, headers)
df_tracks.head(3)

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,[],1,168120,False,{'isrc': 'GBAYE0601510'},{'spotify': 'https://open.spotify.com/track/5e...,https://api.spotify.com/v1/tracks/5e8uc7f0v5jp...,5e8uc7f0v5jpY5SF1emxHl,False,Getting Better - Remastered,0,None,4,track,spotify:track:5e8uc7f0v5jpY5SF1emxHl
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,250160,True,{'isrc': 'USTB10250051'},{'spotify': 'https://open.spotify.com/track/62...,https://api.spotify.com/v1/tracks/62h3HmtlAmot...,62h3HmtlAmot4A9zwKVRSq,False,"Mama, I'm in Love Wit a Gangsta - Mix",26,https://p.scdn.co/mp3-preview/7bfced09b64662fe...,3,track,spotify:track:62h3HmtlAmot4A9zwKVRSq
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,291440,False,{'isrc': 'ES5150103002'},{'spotify': 'https://open.spotify.com/track/0H...,https://api.spotify.com/v1/tracks/0HmQkmfYXRPw...,0HmQkmfYXRPweIg1ycw1R8,False,Morena mía,67,https://p.scdn.co/mp3-preview/3164e42e1f32003f...,8,track,spotify:track:0HmQkmfYXRPweIg1ycw1R8


In [12]:
# create a dictionary of artist + track ids
artist_track_id_dict = {}
for i in range(len(df_tracks.artists)):
    x = df_tracks.artists[i][0]['id']
    y = df_tracks.id[i]
    artist_track_id_dict[x] = y
    
art_track_ids = pd.DataFrame([artist_track_id_dict]).transpose()
art_track_ids = art_track_ids.reset_index()
art_track_ids = art_track_ids.rename(columns={'index': 'artist_id', 0:'track_id'})
art_track_ids.head(3)

,artist_id,track_id
0,3WrFJ7ztbogyGnTHbHJFl2,19aATlcb67bbIdjcMA0rOa
1,3y24n3XhZ96wgwRXjvS17T,5QMTmFY0KC0RfyRbQBEzzE
2,7mWCSSOYqm4E9mB7V4ot6S,2kpHaB82qVWd4ccYXGt5BE


In [13]:
art_ids = art_track_ids.artist_id.to_list()
df_artists = dc.song_attributes('artists/', art_ids, headers)
df_artists.head(3)

,external_urls,followers,genres,href,id,images,name,popularity,type,uri
0,{'spotify': 'https://open.spotify.com/artist/3...,"{'href': None, 'total': 20981982}","[beatlesque, british invasion, classic rock, m...",https://api.spotify.com/v1/artists/3WrFJ7ztbog...,3WrFJ7ztbogyGnTHbHJFl2,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",The Beatles,88,artist,spotify:artist:3WrFJ7ztbogyGnTHbHJFl2
1,{'spotify': 'https://open.spotify.com/artist/3...,"{'href': None, 'total': 526798}","[g funk, gangster rap, hip hop, rap, west coas...",https://api.spotify.com/v1/artists/3y24n3XhZ96...,3y24n3XhZ96wgwRXjvS17T,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Coolio,72,artist,spotify:artist:3y24n3XhZ96wgwRXjvS17T
2,{'spotify': 'https://open.spotify.com/artist/7...,"{'href': None, 'total': 1784540}","[latin, latin pop, mexican pop, rock en espano...",https://api.spotify.com/v1/artists/7mWCSSOYqm4...,7mWCSSOYqm4E9mB7V4ot6S,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Miguel Bosé,72,artist,spotify:artist:7mWCSSOYqm4E9mB7V4ot6S


In [14]:
df_genres = df_artists[['id', 'genres']]
df_genres = df_genres.rename(columns={'id': 'artist_id'})
df_genres.head(3) #explode or 'df_stream.genres.apply(pd.Series)'?

,artist_id,genres
0,3WrFJ7ztbogyGnTHbHJFl2,"[beatlesque, british invasion, classic rock, m..."
1,3y24n3XhZ96wgwRXjvS17T,"[g funk, gangster rap, hip hop, rap, west coas..."
2,7mWCSSOYqm4E9mB7V4ot6S,"[latin, latin pop, mexican pop, rock en espano..."


# 4) Consolidate All DataFrames

### Merge all 4 dataframes: stream, features, track and genres.

In [15]:
df_stream = df_stream.merge(df_features, how='inner', 
                              left_on=['uri'],
                              right_on=['id'])

In [20]:
df_stream = df_stream.merge(art_track_ids, how='inner',
                           left_on=['id'],
                           right_on=['track_id'])

In [21]:
df_stream = df_stream.merge(df_genres, how='inner',
                           left_on='artist_id',
                           right_on='artist_id')

### Return a final dataframe with the columns I will use.

In [23]:
df_stream = df_stream[['endTime','year','month','day','hour',
                       'artist','album','track','genres',
                       'minutesPlayed','id','danceability',
                       'energy','key','loudness','mode',
                       'speechiness','acousticness','instrumentalness',
                       'liveness','valence','tempo']]

In [26]:
df_stream.head(1)

,endTime,year,month,day,hour,artist,album,track,genres,minutesPlayed,...,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,2019-08-29 16:31:00,2019,8,29,16,The Outfield,Super Hits,Your Love,"[album rock, mellow gold, new romantic, new wa...",3.61,...,0.757,1,-7.868,0,0.0601,0.103,0.000001,0.0631,0.582,129.607


### Artist + Genres dataframe

In [27]:
df_genres = df_stream[['artist', 'genres']].explode('genres')
df_genres.head(1)

,artist,genres
0,The Outfield,album rock


# 5) Export New Databases

In [28]:
df_stream.to_csv('data/stream.csv')
df_genres.to_csv('data/genres.csv')